In [1]:
!pwd
%cd ../IndQNER
!pwd

/local/upb/users/a/asepff/profiles/unix/cs/E-IndQNER
/local/upb/users/a/asepff/profiles/unix/cs/IndQNER
/local/upb/users/a/asepff/profiles/unix/cs/IndQNER


/upb/users/a/asepff/profiles/unix/cs/.conda/envs/dice/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
f = open("test.txt", "r")
sentences = f.readlines()
for sentence in sentences:
    print(sentence.strip())
f.close()

Tidak ada sesuatu yang lebih kecil dan yang lebih besar daripada itu, kecuali semua tercatat dalam kitab yang nyata (Lauh Mahfuz).
Ketahuilah bahwa sesungguhnya (bagi) para wali Allah itu tidak ada rasa takut yang menimpa mereka dan mereka pun tidak bersedih.
( Mereka adalah) orang-orang yang beriman dan selalu bertakwa.
Bagi mereka berita gembira di dalam kehidupan di dunia dan di akhirat.
Tidak ada perubahan bagi kalimat-kalimat (ketetapan dan janji) Allah.
Demikian itulah kemenangan yang agung.
Janganlah engkau (Nabi Muhammad) sedih oleh perkataan mereka.
Sesungguhnya kekuasaan itu seluruhnya milik Allah.
Dia Maha Mendengar lagi Maha Mengetahui.
Ketahuilah bahwa sesungguhnya milik Allahlah siapa yang ada di langit dan siapa yang ada di bumi.
Orang-orang yang menyeru sekutu-sekutu selain Allah tidaklah mengikuti (suatu kebenaran).
Mereka hanya mengikuti persangkaan belaka dan mereka hanyalah menduga-duga.
Dialah yang menjadikan malam bagimu agar kamu beristirahat padanya dan (menjadi

In [3]:
import time
import logging
from transformers import logging as hf_logging
import torch
import time
from transformers import BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
import torch
import re
import ast
import pandas as pd
import ast
from accelerate import Accelerator
from collections import deque
from rdflib import Graph
import json
from tqdm import tqdm


base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

device_map = {"": 0}
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

model = AutoModelForCausalLM.from_pretrained(  ## If it fails at this line, restart the runtime and try again.
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True,
    low_cpu_mem_usage=True
)
model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
model.config.pretraining_tp = 1

def generate_answer(pipe, example):
    
    prompt = pipe.tokenizer.apply_chat_template(example["messages"][:2],
                                                tokenize=False,
                                                add_generation_prompt=True)
    terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
    
    outputs = pipe(prompt,
                max_new_tokens=1024,
                eos_token_id=terminators,
                do_sample=True,
                temperature=0.3,
                top_k=30,
                top_p=0.85,
                )
    generated_text = outputs[0]['generated_text']
    return {"content": example["messages"][1]['content'], "generated_text": generated_text}

def create_input_prompt(system_message, user_prompt):
    return {
        "messages": [
            {"role": "system","content": system_message},
            {"role": "user", "content": user_prompt},
        ]
    }

/upb/users/a/asepff/profiles/unix/cs/.conda/envs/dice/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
import json

# Open and read the JSON file
with open("named_entity_class_dictionary.json", "r") as json_file:
    named_entity_classes_dict = json.load(json_file)

# Print the loaded dictionary
print(named_entity_classes_dict)
named_entity_classes = [named_entity_class for named_entity_class in named_entity_classes_dict]
print(named_entity_classes)


{'O': {'en': 'O'}, 'HolyBook': {'en': 'Holy book'}, 'Messenger': {'en': 'Messenger'}, 'Allah': {'en': 'Allah'}, 'Event': {'en': 'Event'}, 'AstronomicalBody': {'en': 'Astronomical body'}, 'Prophet': {'en': 'Prophet'}, 'Person': {'en': 'Person'}, 'Food': {'en': 'Food'}, 'GeographicalLocation': {'en': 'Geographical location'}, 'Color': {'en': 'Color'}, 'Angel': {'en': 'Angel'}, 'Artifact': {'en': 'Artifact'}, 'Religion': {'en': 'Religion'}, 'AfterlifeLocation': {'en': 'Afterlife location'}, 'Throne': {'en': 'Throne'}, 'Language': {'en': 'Language'}}
['O', 'HolyBook', 'Messenger', 'Allah', 'Event', 'AstronomicalBody', 'Prophet', 'Person', 'Food', 'GeographicalLocation', 'Color', 'Angel', 'Artifact', 'Religion', 'AfterlifeLocation', 'Throne', 'Language']


In [5]:
predict_dict = dict()
num = 0
for sentence in sentences:
    num +=1
    system_message = f"""
    Given the following entity classes and sentences, extract entity mentions and their respective class from the sentence according to the sentences' content and the entity classes. 
    In the output, only include entity mentions and their respective class in the given output format. No needed further explanation.
    CONTEXT: entity classes: {named_entity_classes}. 
    Example sentence: Jika kamu (tetap) dalam keraguan tentang apa (Al-Qur’an) yang Kami turunkan kepada hamba Kami (Nabi Muhammad), buatlah satu surah yang semisal dengannya dan ajaklah penolong-penolongmu selain Allah, jika kamu orang-orang yang benar.
    Example output: Jika/O kamu/O (/O tetap/O )/O dalam/O keraguan/O tentang/O apa/O (/O Al-Qur’an/HolyBook )/O yang/O Kami/O turunkan/O kepada/O hamba/O Kami/O (/O Nabi/O Muhammad/Messenger )/O ,/O buatlah/O satu/O surah/O yang/O semisal/O dengannya/O dan/O ajaklah/O penolong-penolongmu/O selain/O Allah/Allah ,/O jika/O kamu/O orang-orang/O yang/O benar/O ./O
    """
    
    user_prompt = f"""
    Test sentence: {sentence}
    Test output:
    """
    
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer) 
    # Generate answers for the dataset
    message = create_input_prompt(system_message, user_prompt)
    results = generate_answer(pipe, message)
    outputs = results["generated_text"]
    select_outputs = outputs.split("\n\n")[-1]
    
    print("##############")
    print(num)
    print(select_outputs)
    print("###############")
    predict_dict[num] = select_outputs

##############
1
Tidak/O ada/O (/O sesuatu/O )/O yang/O lebih/O kecil/O dan/O yang/O lebih/O besar/O daripada/O itu/O,/O kecuali/O semua/O tercatat/O dalam/O kitab/O yang/O nyata/O (/O Lauh/O Mahfuz/HolyBook )/O./O
###############
##############
2
Ketahuilah/O bahwa/O sesungguhnya/O (/O )/O bagi/O para/O wali/O Allah/O tidak/O ada/O rasa/O takut/O yang/O menimpa/O mereka/O dan/O mereka/O pun/O tidak/O bersedih/O./O
###############
##############
3
(Mereka/O )/O adalah/O orang-orang/O yang/O beriman/O dan/O selalu/O bertakwa/O./O
###############
##############
4
Bagi/O mereka/O,/O berita/O gembira/O di/O dalam/O kehidupan/O di/O (/O dunia/GeographicalLocation )/O dan/O di/O akhirat/AfterlifeLocation./O
###############
##############
5
Tidak/O ada/O perubahan/O bagi/O kalimat-kalimat/O (/O ketetapan/O dan/O janji/O )/O Allah/Allah./O
###############
##############
6
Demikian/O itulah/O kemenangan/O yang/O agung/O
###############
##############
7
Janganlah/O engkau/O (/O Nabi/O Muhammad/M

In [6]:
import json
# Save the dictionary to a JSON file
with open("results-zeroshot-attempt-1.json", "w") as json_file:
    json.dump(predict_dict, json_file, indent=4)  # 'indent' adds formatting for readability

In [7]:
import json
import pandas as pd

# Re-loading the new test labels and results
test_labels_path_new = 'test_with_labels.txt'
results_path_new = 'results-zeroshot-attempt-1.json'

# Reading the new test labels
with open(test_labels_path_new, 'r', encoding='utf-8') as f:
    test_labels_new = f.readlines()

# Reading the new results
with open(results_path_new, 'r', encoding='utf-8') as f:
    results_new = json.load(f)

# Extract predictions and test labels into lists for comparison
true_labels_new = []
predicted_labels_new = []

# Assuming test_labels_new contains labels in the form of 'word/label' and results_new is a dictionary with line-by-line labels
for i, line in enumerate(test_labels_new):
    # Extracting labels from the test set
    true_labels_line_new = [token.split('/')[-1] for token in line.strip().split()]
    true_labels_new.extend(true_labels_line_new)

    # Extracting predicted labels from the results
    if str(i+1) in results_new:
        predicted_labels_line_new = [token.split('/')[-1] for token in results_new[str(i+1)].strip().split()]
    else:
        predicted_labels_line_new = ['O'] * len(true_labels_line_new)  # Assuming 'O' if no prediction available

    predicted_labels_new.extend(predicted_labels_line_new)

# Checking lengths of true and predicted labels
len_true_labels_new, len_predicted_labels_new = len(true_labels_new), len(predicted_labels_new)
len_true_labels_new, len_predicted_labels_new


(6056, 5716)

In [10]:
true_labels_new

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'Allah',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'Allah',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'Messenger',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'Allah',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'AstronomicalBody',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'Allah',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 

In [8]:
# Let's trim both lists to the shorter length to align the predictions and true labels
min_length_new = min(len_true_labels_new, len_predicted_labels_new)

true_labels_trimmed_new = true_labels_new[:min_length_new]
predicted_labels_trimmed_new = predicted_labels_new[:min_length_new]

# Now calculate precision, recall, and F1 score
from sklearn.metrics import precision_score, recall_score, f1_score

precision_new = precision_score(true_labels_trimmed_new, predicted_labels_trimmed_new, average='weighted', zero_division=0)
recall_new = recall_score(true_labels_trimmed_new, predicted_labels_trimmed_new, average='weighted', zero_division=0)
f1_new = f1_score(true_labels_trimmed_new, predicted_labels_trimmed_new, average='weighted', zero_division=0)

precision_new, recall_new, f1_new


(0.9158679050030459, 0.8950314905528342, 0.9053037063131885)

In [9]:
true_labels_trimmed_new

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'Allah',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'Allah',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'Messenger',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'Allah',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'AstronomicalBody',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'Allah',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 